In [1]:
# load Python modules
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')

In [2]:
### load data
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print('train size:', train_df.shape)
print('test size:', test_df.shape)
print('have the same columns?', all(train_df.drop('target', axis=1).columns == test_df.columns))
train_df_org = train_df
test_df_org = test_df

train size: (595212, 59)
test size: (892816, 58)
have the same columns? True


In [3]:
###data cleansing
# remove duplicatives if exists
# wrt rows
train_df = train_df.drop_duplicates()
# wrt columns (get recursion error)
#train_df = train_df.T.drop_duplicates().T

rows = train_df.shape[0]
columns = train_df.shape[1]
print("The train dataset contains {0} rows and {1} columns".format(rows, columns))
test_df.head()

The train dataset contains 595212 rows and 59 columns


,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1


In [4]:
# remove constant values
train_df = train_df.loc[:, (train_df != train_df.iloc[0]).any()]
test_df = test_df.loc[:, train_df.drop('target', axis=1).columns]

In [5]:
# fill nan by median
train_df = train_df.replace(-1, np.NaN)
test_df = test_df.replace(-1, np.NaN)
print('nan exists in train?:', train_df.isnull().any().any())
print('nan exists in test?:', test_df.isnull().any().any())
train_median = train_df.drop('target', axis=1).median()
train_df = train_df.fillna(train_median)
test_df = test_df.fillna(train_median)

nan exists in train?: True
nan exists in test?: True


In [6]:
# separate data
train_y = train_df.loc[:, 'target']
train_id = train_df.loc[:, 'id']
train_df = train_df.drop(['target', 'id'], axis=1)
train_df_float = train_df.select_dtypes(include=['float64'])
train_df_int = train_df.select_dtypes(include=['int64'])
test_id = test_df.loc[:, 'id']
test_df = test_df.drop('id', axis=1)
test_df_float = test_df.select_dtypes(include=['float64'])
test_df_int = test_df.select_dtypes(include=['int64'])
print('train float:', len(train_df_float.columns))
print('train int:', len(train_df_int.columns))
print('test float:', len(test_df_float.columns))
print('test int:', len(test_df_int.columns))

train float: 20
train int: 37
test float: 20
test int: 37


In [7]:
# normalize data
train_df_float_mean = train_df_float.mean()
train_df_float_std = train_df_float.std()
train_df_float_norm = (train_df_float - train_df_float_mean) / (train_df_float_std + 1.e-9)
test_df_float_norm = (test_df_float - train_df_float_mean) / (train_df_float_std + 1.e-9)

train_df_norm = pd.concat((train_df_float_norm, train_df_int), axis=1)
test_df_norm = pd.concat((test_df_float_norm, test_df_int), axis=1)
print(train_df_norm.shape)
print(test_df_norm.shape)

(595212, 57)
(892816, 57)


In [8]:
### learn xgboost w/ default parameters
xgb_model = xgb.XGBRegressor()
xgb_model.fit(train_df_norm, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
predict_y = xgb_model.predict(test_df_norm)

In [10]:
predict_submit = pd.concat((test_id, pd.DataFrame(data=predict_y, columns=['target'])), axis=1)
predict_submit.head()

,id,target
0,0,0.030289
1,1,0.026481
2,2,0.030233
3,3,0.014275
4,4,0.034863


In [11]:
# save csv
predict_submit.to_csv('./xgb_submission.csv', index=False)